In [1]:
import gymnasium as gym
from gymnasium import envs

# List all registered environments (including Atari)
all_envs = envs.registry.keys()
atari_envs = [env_id for env_id in all_envs if "ALE/" in env_id]

# Print first 10 Atari games
print(list(atari_envs)[:10])  # Should include 'ALE/Frogger-v5'

[]


In [2]:
import gymnasium as gym
import os
import numpy as np
import pandas as pd
import seaborn as sns
import ale_py

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
'''
import gymnasium as gym

# Modern way to create Atari environment
env = gym.make("ALE/Frogger-v5", render_mode="human")  # Note: ALE/ prefix required


observation, info = env.reset()
for _ in range(100):
    action = env.action_space.sample()  # Random action
    observation, reward, terminated, truncated, info = env.step(action)
        
    if terminated or truncated:
        observation, info = env.reset()
'''

'\nimport gymnasium as gym\n\n# Modern way to create Atari environment\nenv = gym.make("ALE/Frogger-v5", render_mode="human")  # Note: ALE/ prefix required\n\n\nobservation, info = env.reset()\nfor _ in range(100):\n    action = env.action_space.sample()  # Random action\n    observation, reward, terminated, truncated, info = env.step(action)\n        \n    if terminated or truncated:\n        observation, info = env.reset()\n'

In [4]:
import gymnasium as gym
import numpy as np
from IPython.display import clear_output
import random

# Initialize environment
env = gym.make("ALE/Frogger-v5", render_mode="rgb_array", obs_type="grayscale")  # Simplified observation

def discretize_observation(obs, bins):
    """Process 210x160 grayscale image into simplified state"""
    # Find frog position (bright pixel in bottom area)
    frog_area = obs[180:200, 70:90]  # Approximate frog spawn area
    frog_y, frog_x = np.unravel_index(np.argmax(frog_area), frog_area.shape)
    
    # Find closest lane (horizontal line detection)
    lanes = [50, 75, 100, 125]  # Approximate lane positions
    current_lane = np.argmin([abs(frog_y - lane) for lane in lanes])
    
    # Discretize
    y_bin = min(int(frog_y * bins[0] / 210), bins[0]-1)
    lane_bin = min(current_lane, bins[1]-1)
    return (y_bin, lane_bin)

# Q-table setup
obs_space_size = (10, 4)  # (y-position bins, lane bins)
action_space_size = env.action_space.n
qtable = np.zeros((*obs_space_size, action_space_size))

# Rest of your Q-learning code...
# Q-table setup
obs_space_size = (10, 10)  # Simplified state space
action_space_size = env.action_space.n
qtable = np.zeros((*obs_space_size, action_space_size))

# Hyperparameters
alpha = 0.1
gamma = 0.99
epsilon = 0.1

# Tracking variables
total_episodes = 1000
successful_episodes = 0
total_steps = 0

for episode in range(total_episodes):
    state = discretize_observation(env.reset()[0], obs_space_size)
    done = False
    steps = 0

    while not done:
        # Epsilon-greedy action selection
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Random action
        else:
            action = np.argmax(qtable[state])  # Best Q-value action

        new_state, reward, terminated, truncated, _ = env.step(action)
        new_state = discretize_observation(new_state, obs_space_size)
        done = terminated or truncated

        # Q-table update
        qtable[state][action] = (1 - alpha) * qtable[state][action] + alpha * (
            reward + gamma * np.max(qtable[new_state]))
        
        state = new_state  # This line was likely misaligned in your original code
        steps += 1

    total_steps += steps
    if reward > 0:  # Positive reward for successful crossing
        successful_episodes += 1

    if episode % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {episode}, Success Rate: {successful_episodes/(episode+1):.2f}")

env.close()

# Results
success_rate = successful_episodes / total_episodes
print(f"\nFinal Success Rate: {success_rate * 100:.2f}%")
print(f"Average Steps per Episode: {total_steps / total_episodes:.2f}")

Episode: 0, Success Rate: 0.00


KeyboardInterrupt: 